In [1]:
%pylab inline
import re
import math
import string
import pandas as pd
import csv
from collections import Counter
from __future__ import division

Populating the interactive namespace from numpy and matplotlib


In [2]:
TEXT = open('data/big.txt').read()

In [3]:
def tokens(text):
    return re.findall('[a-z]+', text.lower())

In [4]:
WORDS = tokens(TEXT)
COUNTS_BIG = Counter(WORDS)

with open('count_1w.csv', mode='r') as counts_1:
    reader = csv.reader(counts_1)
    for rows in reader:
        mydict = {rows[0]:int(rows[1]) for rows in reader}
COUNTS_SPELL = Counter(mydict)

In [5]:
COUNTS = COUNTS_BIG
print(COUNTS.most_common(10))

[('the', 80030), ('of', 40025), ('and', 38313), ('to', 28766), ('in', 22050), ('a', 21155), ('that', 12512), ('he', 12401), ('was', 11410), ('it', 10681)]


In [6]:
spell = open('data/spell-errors.txt','r')
line = spell.readline()
spell_errors = []
while line:
    spell_errors.append(re.findall('[a-z]+', line.lower()))
    line = spell.readline()
spell.close()

In [7]:
def correct(word):
    candidates = (known(edits0(word)) or 
                  known(edits1(word)) or 
                  known(edits2(word)) or 
                  [word])
    return max(candidates, key=COUNTS.get)

In [8]:
def known(words):
    #Return the subset of words that are actually in the dictionary."
    return {w for w in words if w in COUNTS}

def edits0(word): 
    return {word}

def edits2(word):
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

In [9]:
def edits1(word):
    pairs      = splits(word)
    deletes    = [a+b[1:]           for a, b in pairs if b]
    transposes = [a+b[1]+b[0]+b[2:] for a, b in pairs if len(b) > 1]
    replaces   = [a+c+b[1:]         for a, b in pairs for c in alphabet if b]
    inserts    = [a+c+b             for a, b in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

def splits(word):
    return [(word[:i], word[i:]) 
            for i in range(len(word)+1)]

alphabet = 'abcdefghijklmnopqrstuvwxyz'

In [10]:
def correct_text(text):
    return re.sub('[a-zA-Z]+', correct_match, text)

def correct_match(match):
    word = match.group()
    return case_of(word)(correct(word.lower()))

def case_of(text):
    return (str.upper if text.isupper() else
            str.lower if text.islower() else
            str.title if text.istitle() else
            str)

In [11]:
def correct_spell_err(text):
    return re.sub('[a-zA-Z]+', correct_matcher, text)

def correct_matcher(match):
    word = match.group()
    return case_of1(word)(prime_check(word.lower()))

def case_of1(text):
    return (str.upper if text.isupper() else
            str.lower if text.islower() else
            str.title if text.istitle() else
            str)

In [12]:
def prime_check(word):
    for i in range(len(spell_errors)):
        for j in range(len(spell_errors[i])):
            if(spell_errors[i][j] == word):
                return spell_errors[i][0]
    return word

In [13]:
test = pd.read_csv("test.csv")
x_test = []
for index, row in test.iterrows():
    check = correct_spell_err(row['WRONG'])
    if check == row['WRONG']:
        check = correct_text(row['WRONG'])
    x_test.append(check)
correct_spell = pd.DataFrame()
correct_spell['CORRECT'] = x_test
writer = pd.ExcelWriter('data/test_darshil.xlsx')
correct_spell.to_excel(writer,'sheet1')
writer.save()